<a href="https://colab.research.google.com/github/maxhormazabal/depencendy_parsing/blob/main/P2_Models_NLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos

In [ ]:
# Getting access to Google Drive files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/MyDrive/MASTER")
from nlu_model_utils import *
import tensorflow as tf
from keras import Input, Model
from keras.layers import Dense
import pandas as pd
import numpy as np
import math

In [ ]:
stack_len = 2
buffer_len = 3

path = "nlu_data/"+str(stack_len)+"stack"+str(buffer_len)+"buffer"

(x_train_stack,x_train_buffer,x_train_stack_upos,x_train_buffer_upos,action_train,deprel_train) = getDataSource('train',folder=path)
(x_test_stack,x_test_buffer,x_test_stack_upos,x_test_buffer_upos,action_test,deprel_test) = getDataSource('test',folder=path)
(x_val_stack,x_val_buffer,x_val_stack_upos,x_val_buffer_upos,action_val,deprel_val) = getDataSource('val',folder=path)

In [ ]:
action_shape = action_train[0].shape[0]
deprel_shape = deprel_train[0].shape[0]

input1 = tf.keras.layers.Input(shape=(stack_len,),name = 'Stack_Input')
input2 = tf.keras.layers.Input(shape=(buffer_len,),name = 'Buffer_Input')
merged = tf.keras.layers.Concatenate(axis=1,name = 'Concat_Layer')([input1, input2])
dense1 = tf.keras.layers.Dense(3, input_dim=2, activation='sigmoid', use_bias=True,name = 'Dense_Layer')(merged)
output1 = tf.keras.layers.Dense(action_shape, activation='softmax', use_bias=True,name = 'Action_Output')(dense1)
output2 = tf.keras.layers.Dense(deprel_shape, activation='softmax', use_bias=True,name = 'Deprel_Output')(dense1)
model = tf.keras.Model(inputs=[input1,input2],outputs=[output1,output2])

opt = tf.keras.optimizers.Adam(learning_rate=0.01) #default 0.001
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
print(model.summary())

model.fit([x_train_stack,x_train_buffer],[action_train, deprel_train], epochs=10, batch_size=100)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Stack_Input (InputLayer)       [(None, 2)]          0           []                               
                                                                                                  
 Buffer_Input (InputLayer)      [(None, 3)]          0           []                               
                                                                                                  
 Concat_Layer (Concatenate)     (None, 5)            0           ['Stack_Input[0][0]',            
                                                                  'Buffer_Input[0][0]']           
                                                                                                  
 Dense_Layer (Dense)            (None, 3)            18          ['Concat_Layer[0][0]']     

In [ ]:
print(x_test_stack.shape,x_test_buffer.shape,action_test.shape,deprel_test.shape)

(6969, 2) (6969, 3) (6969, 6) (6969, 34)


In [ ]:
print(x_test_stack[:1],x_test_buffer[:1],np.argmax(action_train[:1]),np.argmax(deprel_train[:1]))

[[   0. 3157.]] [[2111.   14. 2112.]] 3 0


In [ ]:
x_test_stack[:1]

array([[   0., 3157.]], dtype=float32)

In [ ]:
# print("Evaluate model on test data")
# results = model.evaluate([x_test_stack,x_test_buffer],[action_test,deprel_test], batch_size=128)
# print("test loss, test acc:", results)

# Generate a prediction using model.predict() 
# and calculate it's shape:
print("Generate a prediction")
prediction = model.predict([x_test_stack[:1],x_test_buffer[:1]])
# print("prediction shape:", prediction.shape)

Generate a prediction
1/1 [==============================] - 0s 25ms/step


In [ ]:
number2action[np.argmax(prediction[0][0])]
number2deprel_test[np.argmax(prediction[1][0])]

'None'

In [ ]:
np.argmax(prediction[1][0])

0

In [ ]:
print("Los input: ",x_test_stack[:1],x_test_buffer[:1],
      "Entregan como resultado: ",number2action[np.argmax(prediction[0][0])],number2deprel_test[np.argmax(prediction[1][0])])

Los input:  [[   0. 3157.]] [[2111.   14. 2112.]] Entregan como resultado:  Shift None


In [ ]:
number2action = {
  1: 'Right Arc', 
  2: 'Left Arc', 
  3: 'Shift', 
  4: 'Reduce', 
  5: 'Done'
}

number2deprel_test = {
  0: 'None',
  1: 'acl',
  2: 'acl:relcl',
  3: 'advcl',
  4: 'advmod',
  5: 'amod',
  6: 'appos',
  7: 'aux',
  8: 'aux:pass',
  9: 'case',
  10: 'cc',
  11: 'ccomp',
  12: 'compound',
  13: 'compound:prt',
  14: 'conj',
  15: 'cop',
  16: 'det',
  17: 'det:predet',
  18: 'expl',
  19: 'fixed',
  20: 'flat',
  21: 'flat:foreign',
  22: 'iobj',
  23: 'mark',
  24: 'nmod',
  25: 'nmod:poss',
  26: 'nsubj',
  27: 'nsubj:pass',
  28: 'nummod',
  29: 'obj',
  30: 'obl',
  31: 'punct',
  32: 'root',
  33: 'xcomp'
}

In [ ]:
stack_len = 4
buffer_len = 8

path = "nlu_data/"+str(stack_len)+"stack"+str(buffer_len)+"buffer"

(x_train_stack,x_train_buffer,x_train_stack_upos,x_train_buffer_upos,action_train,deprel_train) = getDataSource('train',folder=path)
(x_test_stack,x_test_buffer,x_test_stack_upos,x_test_buffer_upos,action_test,deprel_test) = getDataSource('test',folder=path)
(x_val_stack,x_val_buffer,x_val_stack_upos,x_val_buffer_upos,action_val,deprel_val) = getDataSource('val',folder=path)

In [ ]:
action_shape = action_train[0].shape[0]
deprel_shape = deprel_train[0].shape[0]

input1 = tf.keras.layers.Input(shape=(stack_len,),name = 'Stack_Input')
input2 = tf.keras.layers.Input(shape=(buffer_len,),name = 'Buffer_Input')
merged = tf.keras.layers.Concatenate(axis=1,name = 'Concat_Layer')([input1, input2])
dense1 = tf.keras.layers.Dense(3, input_dim=2, activation='sigmoid', use_bias=True,name = 'Dense_Layer')(merged)
output1 = tf.keras.layers.Dense(action_shape, activation='softmax', use_bias=True,name = 'Action_Output')(dense1)
output2 = tf.keras.layers.Dense(deprel_shape, activation='softmax', use_bias=True,name = 'Deprel_Output')(dense1)
model = tf.keras.Model(inputs=[input1,input2],outputs=[output1,output2])

opt = tf.keras.optimizers.Adam(learning_rate=0.01) #default 0.001
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
print(model.summary())

model.fit([x_train_stack,x_train_buffer],[action_train, deprel_train], epochs=10, batch_size=100)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Stack_Input (InputLayer)       [(None, 4)]          0           []                               
                                                                                                  
 Buffer_Input (InputLayer)      [(None, 8)]          0           []                               
                                                                                                  
 Concat_Layer (Concatenate)     (None, 12)           0           ['Stack_Input[0][0]',            
                                                                  'Buffer_Input[0][0]']           
                                                                                                  
 Dense_Layer (Dense)            (None, 3)            39          ['Concat_Layer[0][0]']     

In [ ]:
stack_len = 10
buffer_len = 15

path = "nlu_data/"+str(stack_len)+"stack"+str(buffer_len)+"buffer"

(x_train_stack,x_train_buffer,x_train_stack_upos,x_train_buffer_upos,action_train,deprel_train) = getDataSource('train',folder=path)
(x_test_stack,x_test_buffer,x_test_stack_upos,x_test_buffer_upos,action_test,deprel_test) = getDataSource('test',folder=path)
(x_val_stack,x_val_buffer,x_val_stack_upos,x_val_buffer_upos,action_val,deprel_val) = getDataSource('val',folder=path)

In [ ]:
action_shape = action_train[0].shape[0]
deprel_shape = deprel_train[0].shape[0]

input1 = tf.keras.layers.Input(shape=(stack_len,),name = 'Stack_Input')
input2 = tf.keras.layers.Input(shape=(buffer_len,),name = 'Buffer_Input')
merged = tf.keras.layers.Concatenate(axis=1,name = 'Concat_Layer')([input1, input2])
dense1 = tf.keras.layers.Dense(3, input_dim=2, activation='sigmoid', use_bias=True,name = 'Dense_Layer')(merged)
output1 = tf.keras.layers.Dense(action_shape, activation='softmax', use_bias=True,name = 'Action_Output')(dense1)
output2 = tf.keras.layers.Dense(deprel_shape, activation='softmax', use_bias=True,name = 'Deprel_Output')(dense1)
model = tf.keras.Model(inputs=[input1,input2],outputs=[output1,output2])

opt = tf.keras.optimizers.Adam(learning_rate=0.01) #default 0.001
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
print(model.summary())

model.fit([x_train_stack,x_train_buffer],[action_train, deprel_train], epochs=10, batch_size=100)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Stack_Input (InputLayer)       [(None, 4)]          0           []                               
                                                                                                  
 Buffer_Input (InputLayer)      [(None, 8)]          0           []                               
                                                                                                  
 Concat_Layer (Concatenate)     (None, 12)           0           ['Stack_Input[0][0]',            
                                                                  'Buffer_Input[0][0]']           
                                                                                                  
 Dense_Layer (Dense)            (None, 3)            39          ['Concat_Layer[0][0]']     

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01) #default 0.001
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
print(model.summary())

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Stack_Input (InputLayer)       [(None, 2)]          0           []                               
                                                                                                  
 Buffer_Input (InputLayer)      [(None, 3)]          0           []                               
                                                                                                  
 Concat_Layer (Concatenate)     (None, 5)            0           ['Stack_Input[0][0]',            
                                                                  'Buffer_Input[0][0]']           
                                                                                                  
 Dense_Layer (Dense)            (None, 3)            18          ['Concat_Layer[0][0]']     

In [ ]:
model.fit([empty_stack,empty_buffer],[action_train, deprel_train], epochs=10, batch_size=100)

Epoch 1/10
888/888 [==============================] - 3s 2ms/step - loss: 2.5672 - Action_Output_loss: 1.1339 - Deprel_Output_loss: 1.4334 - Action_Output_accuracy: 0.5040 - Deprel_Output_accuracy: 0.7112
Epoch 2/10
888/888 [==============================] - 2s 2ms/step - loss: 2.0447 - Action_Output_loss: 0.8157 - Deprel_Output_loss: 1.2291 - Action_Output_accuracy: 0.6816 - Deprel_Output_accuracy: 0.7233
Epoch 3/10
888/888 [==============================] - 2s 2ms/step - loss: 1.9622 - Action_Output_loss: 0.7581 - Deprel_Output_loss: 1.2041 - Action_Output_accuracy: 0.6920 - Deprel_Output_accuracy: 0.7233
Epoch 4/10
888/888 [==============================] - 2s 2ms/step - loss: 1.9208 - Action_Output_loss: 0.7244 - Deprel_Output_loss: 1.1963 - Action_Output_accuracy: 0.7022 - Deprel_Output_accuracy: 0.7233
Epoch 5/10
888/888 [==============================] - 2s 2ms/step - loss: 1.8909 - Action_Output_loss: 0.7037 - Deprel_Output_loss: 1.1872 - Action_Output_accuracy: 0.7044 - Deprel

In [ ]:
stack_shape = empty_stack.shape[1]
buffer_shape = empty_buffer.shape[1]

action_shape = action_train[0].shape[0]
deprel_shape = deprel_train[0].shape[0]

input1 = tf.keras.layers.Input(shape=(stack_shape,),name = 'Stack_Input')
input2 = tf.keras.layers.Input(shape=(buffer_shape,),name = 'Buffer_Input')
merged = tf.keras.layers.Concatenate(axis=1,name = 'Concat_Layer')([input1, input2])
dense1 = tf.keras.layers.Dense(15, input_dim=2, activation='sigmoid', use_bias=True,name = 'Dense_Layer')(merged)
dense2 = tf.keras.layers.Dense(30, activation='relu',name = 'Dense_Layer2')(dense1)
output1 = tf.keras.layers.Dense(action_shape, activation='softmax', use_bias=True,name = 'Action_Output')(dense2)
output2 = tf.keras.layers.Dense(deprel_shape, activation='softmax', use_bias=True,name = 'Deprel_Output')(dense2)
model = tf.keras.Model(inputs=[input1,input2],outputs=[output1,output2])

opt = tf.keras.optimizers.Adam(learning_rate=0.001) #default 0.001
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
print(model.summary())

model.fit([empty_stack,empty_buffer],[action_train, deprel_train], epochs=10, batch_size=100)

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Stack_Input (InputLayer)       [(None, 2)]          0           []                               
                                                                                                  
 Buffer_Input (InputLayer)      [(None, 3)]          0           []                               
                                                                                                  
 Concat_Layer (Concatenate)     (None, 5)            0           ['Stack_Input[0][0]',            
                                                                  'Buffer_Input[0][0]']           
                                                                                                  
 Dense_Layer (Dense)            (None, 15)           90          ['Concat_Layer[0][0]']    

In [ ]:
stack_shape = empty_stack.shape[1]
buffer_shape = empty_buffer.shape[1]

action_shape = action_train[0].shape[0]
deprel_shape = deprel_train[0].shape[0]

input1 = tf.keras.layers.Input(shape=(stack_shape,),name = 'Stack_Input')
input2 = tf.keras.layers.Input(shape=(buffer_shape,),name = 'Buffer_Input')
merged = tf.keras.layers.Concatenate(axis=1,name = 'Concat_Layer')([input1, input2])
dense1 = tf.keras.layers.Dense(15, input_dim=2, activation='sigmoid', use_bias=True,name = 'Dense_Layer')(merged)
dense2 = tf.keras.layers.Dense(30, activation='relu',name = 'Dense_Layer2')(dense1)
output1 = tf.keras.layers.Dense(action_shape, activation='softmax', use_bias=True,name = 'Action_Output')(dense2)
output2 = tf.keras.layers.Dense(deprel_shape, activation='softmax', use_bias=True,name = 'Deprel_Output')(dense2)
model = tf.keras.Model(inputs=[input1,input2],outputs=[output1,output2])

opt = tf.keras.optimizers.SGD(learning_rate=0.001) #default 0.001
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
print(model.summary())

model.fit([empty_stack,empty_buffer],[action_train, deprel_train], epochs=10, batch_size=100)

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Stack_Input (InputLayer)       [(None, 2)]          0           []                               
                                                                                                  
 Buffer_Input (InputLayer)      [(None, 3)]          0           []                               
                                                                                                  
 Concat_Layer (Concatenate)     (None, 5)            0           ['Stack_Input[0][0]',            
                                                                  'Buffer_Input[0][0]']           
                                                                                                  
 Dense_Layer (Dense)            (None, 15)           90          ['Concat_Layer[0][0]']    

In [ ]:
stack_shape = empty_stack.shape[1]
buffer_shape = empty_buffer.shape[1]

action_shape = action_train[0].shape[0]
deprel_shape = deprel_train[0].shape[0]

input1 = tf.keras.layers.Input(shape=(stack_shape,),name = 'Stack_Input')
input2 = tf.keras.layers.Input(shape=(buffer_shape,),name = 'Buffer_Input')
merged = tf.keras.layers.Concatenate(axis=1,name = 'Concat_Layer')([input1, input2])
dense1 = tf.keras.layers.Dense(15, input_dim=2, activation='sigmoid', use_bias=True,name = 'Dense_Layer')(merged)
dense2 = tf.keras.layers.Dense(30, activation='relu',name = 'Dense_Layer2')(dense1)
output1 = tf.keras.layers.Dense(action_shape, activation='softmax', use_bias=True,name = 'Action_Output')(dense2)
output2 = tf.keras.layers.Dense(deprel_shape, activation='softmax', use_bias=True,name = 'Deprel_Output')(dense2)
model = tf.keras.Model(inputs=[input1,input2],outputs=[output1,output2])

opt = tf.keras.optimizers.RMSprop(learning_rate=0.001) #default 0.001
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
print(model.summary())

model.fit([empty_stack,empty_buffer],[action_train, deprel_train], epochs=10, batch_size=100)

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Stack_Input (InputLayer)       [(None, 2)]          0           []                               
                                                                                                  
 Buffer_Input (InputLayer)      [(None, 3)]          0           []                               
                                                                                                  
 Concat_Layer (Concatenate)     (None, 5)            0           ['Stack_Input[0][0]',            
                                                                  'Buffer_Input[0][0]']           
                                                                                                  
 Dense_Layer (Dense)            (None, 15)           90          ['Concat_Layer[0][0]']    

In [ ]:
for index,element in enumerate(x_train_df['Stack']):
  x_train_df.loc[index,'Stack'] = np.asarray(element).astype(np.float32)

for index,element in enumerate(x_train_df['Buffer']):
  x_train_df.loc[index,'Buffer'] = np.asarray(element).astype(np.float32)

In [ ]:
np.asarray(x_train_df['Buffer'].values)

ValueError: ignored

In [ ]:
model.fit([np.asarray(x_train_df['Buffer'].values),x_train_df['Buffer'].values],[action_train, deprel_train], epochs=5, batch_size=500)

ValueError: ignored